# vaccine-doses-on-hand

By [Sean Greene](https://www.latimes.com/people/sean-greene)

Downloads the number of vaccine doses on hand from a Tableau dashboard published by the California Department of Public Health.

## Import

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Import dependencies.

In [2]:
import os
import pytz
from datetime import datetime

In [3]:
from tableauscraper import TableauScraper as TS

## Scrape

Set the URL

In [4]:
host = "https://public.tableau.com"

In [5]:
path = "/views/COVID-19VaccineProviderDashboardPublic/PublicVaccineProviderDashboard"

In [6]:
url = f"{host}{path}"

In [7]:
ts = TS()
ts.loads(url)

In [8]:
dashboard = ts.getWorkbook()

In [9]:
target = "Main Vaccine Data Check (6)"

In [10]:
sheet = next(w for w in dashboard.worksheets if w.name == target)

In [11]:
df = sheet.data

## Export

Save out the data as a CSV that's datestamped to California time.

In [12]:
tz = pytz.timezone("America/Los_Angeles")

In [13]:
today = datetime.now(tz).date()

In [14]:
data_dir = os.path.join(os.path.abspath(""), "data")

In [15]:
df.to_csv(os.path.join(data_dir, f"{today}.csv"), index=False)